In [2]:

import cv2
import numpy as np
def show(image):
    # cv2.imwrite('rocks_within_lava.jpg', rocks_within_lava)
    cv2.imshow('Rocks Within Lava', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

# Read the image
image = cv2.imread('test/lava20.jpg')
show(image)
# Convert the image to HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
blurred_hsv = cv2.GaussianBlur(hsv_image, (21,21), 4) 
# Define a lower and upper range for lava color in HSV
lower_lava = np.array([1,190,160])
upper_lava = np.array([60, 255, 255])

# Create a mask for lava within the specified range
lava_mask = cv2.inRange(hsv_image, lower_lava, upper_lava)
show(lava_mask)
# Apply the lava mask to the original image
lava_only = cv2.bitwise_and(image, image, mask=lava_mask)
show(lava_only)
# Convert the lava-only image to grayscale
gray_lava = cv2.cvtColor(lava_only, cv2.COLOR_BGR2GRAY)
show(gray_lava)

# Apply a threshold to create a binary mask for rocks within the lava region
ret, rock_mask = cv2.threshold(gray_lava, 1, 2, cv2.THRESH_BINARY)
show(rock_mask)
# Apply the rock mask to the original image
rocks_within_lava = cv2.bitwise_and(image, image, mask=rock_mask)

# Save or display the result
show(rocks_within_lava)


In [4]:
contours, _ = cv2.findContours(lava_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Keep only the largest contour
largest_contour = contours[0]
# Draw contours around the water region
cv2.drawContours(image, largest_contour, -1, (0, 255, 0), 2)
show(image)

In [5]:
image = cv2.imread('test/lava21.jpg')
# show(image)
# Convert the image to HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
blurred_hsv = cv2.GaussianBlur(hsv_image, (21,21), 6) 

lower_lava = np.array([2,120,120])
upper_lava = np.array([255,255,255])
# lower_lava = np.array([100,2,2])
# upper_lava = np.array([255, 120, 120])



lava_mask = cv2.inRange(blurred_hsv, lower_lava, upper_lava)
# show(lava_mask)
show(lava_mask)
# Find connected components in the binary image
_, labels, stats, _ = cv2.connectedComponentsWithStats(lava_mask, connectivity=8)

# Find the label corresponding to the largest connected component (excluding background)
largest_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1

# Create a mask for the largest connected component
largest_component_mask = np.uint8(labels == largest_label) * 255
show(largest_component_mask)

# Apply the mask to the original image'"ground truth"
# show(image)
result = cv2.bitwise_and(image, image, mask=largest_component_mask)

# Save or display the result

show(result)

contours, _ = cv2.findContours(largest_component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = sorted(contours, key=cv2.contourArea, reverse=True)

# # Keep only the largest contour
# largest_contour = contours[0]
# Draw contours around the water region
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

# for contour in contours:
#     # Approximate the contour with a polygon
#     epsilon = 0.01 * cv2.arcLength(contour, True)
#     approx = cv2.approxPolyDP(contour, epsilon, True)

#     # Draw the original and approximated contours
#     cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
#     cv2.drawContours(image, [approx], -1, (255, 0, 0), 2)
result_image = np.zeros_like(image)

for contour in contours:
    # Adjust the epsilon value to change the resolution of polygons
    epsilon = 0.003 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Draw the smoothed contour on the result image
    cv2.polylines(result_image, [approx], isClosed=True, color=(255, 255, 255), thickness=2)
    cv2.fillPoly(result_image, [approx], color=(255, 255, 255))

# Draw filled white contours on the black image
# cv2.drawContours(result_image, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

show(result_image)